In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np

from core.database.db import DB
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, bm25_weight
from implicit.bpr import BayesianPersonalizedRanking


from ALSRecommender.metrics import fuzzy, precision
from ALSRecommender.preprocessing import Matrix, filter_old_cards, filter_rare_cards, filter_rare_goods, filter_old_goods, filter_by_quantile

db = DB(db='recsys')

ImportError: cannot import name 'get_seasonality_weekly'

In [4]:
%reload_ext autoreload
%autoreload 2

### Препроцессинг трейна

In [19]:
# train = pd.read_sql('select dates, plu_id::int as "PLU_ID", crd_no::text as "CRD_NO", weight from karusel.train_attenuation', con = db.engine)
train = pd.read_sql('select "CRD_NO"::text, "PLU_ID"::bigint, dates from karusel.train', con = db.engine)
print('Shape: %s' % train.shape[0])

train = filter_rare_goods(train, rarity_num=5)
print('Shape without rare goods: %s' % train.shape[0])

train = filter_rare_cards(train, rarity_num=5)
print('Shape without rare cards: %s' % train.shape[0])

# train = filter_old_cards(train, month_threshold=1)
# print('Shape without old cards: %s' % train.shape[0])

train = filter_old_goods(train, month_threshold=1)
print('Shape without old goods: %s' % train.shape[0])

train = filter_by_quantile(train, plu_count_quantiles=(0, 1), cards_count_quantiles=(0.1, 1))
print('Shape without low and high quantiles: %s' % train.shape[0])

Shape: 17985532
Shape without rare goods: 17591183
Shape without rare cards: 17231447
Shape without old goods: 15040751
Shape without low and high quantiles: 14900323


### Убираем товары, для которых неизвестна категория (таких очень мало, и непонятно почему возникают)

In [20]:
products = pd.read_sql('select * from karusel.products', con = db.engine)
print('Shape before filter: %s'% train.shape[0])
plu_id_unique_products = products['plu_id'].unique()
train = train[train['PLU_ID'].isin(plu_id_unique_products)]
print('Shape after filter: %s'% train.shape[0])

Shape before filter: 14900323
Shape after filter: 14900323


### Конвертируем в матрицу

In [21]:
from ALSRecommender.preprocessing import Matrix

matrix = Matrix(train, 
                aggregation='attenuation', products=products[['level_4_name', 'plu_id']])
# matrix = matrix.transform(method='clip', clip_upper_value=1000)
# matrix = matrix.transform(method='log')
matrix = matrix.apply_weights(weight='bm25')

## Подготовка и очистка тестового сета

In [22]:
test = pd.read_sql('select "CRD_NO"::text, "PLU_ID"::bigint from karusel.test', con = db.engine)
val = pd.read_sql('select "CRD_NO"::text, "PLU_ID"::bigint from karusel.val', con = db.engine)

val = val.groupby('CRD_NO').filter(lambda x: len(x['CRD_NO']) > 5)  
test = test.groupby('CRD_NO').filter(lambda x: len(x['CRD_NO']) > 5)  

test.columns = [x.lower() for x in test.columns]
products.columns = [x.lower() for x in products.columns]
val.columns = [x.lower() for x in val.columns]
products['plu_id'] = products['plu_id']

crd_no_unique_train = matrix.index.unique()
plu_id_unique_train = matrix.columns.unique()
test = test[test['crd_no'].isin(crd_no_unique_train)]
test = test[test['plu_id'].isin(plu_id_unique_train)]
val = val[val['crd_no'].isin(crd_no_unique_train)]
val = val[val['plu_id'].isin(plu_id_unique_train)]

plu_category_dict = products.set_index('plu_id').to_dict()['level_2_name']
val_facts_dict = dict(val[['crd_no', 'plu_id']].groupby('crd_no').apply(lambda x: x['plu_id'].unique().tolist()))
test_facts_dict = dict(test[['crd_no', 'plu_id']].groupby('crd_no').apply(lambda x: x['plu_id'].unique().tolist()))

plu_price = pd.read_sql('select plu_id::integer, mean_price from karusel.plu_price', con=db.engine)
plu_price['mean_price'] = plu_price['mean_price'].astype('float16')
plu_price = dict(plu_price[['plu_id', 'mean_price']].values.tolist())

### Строим модель

In [31]:
model = ItemItemRecommender(K=5)  

model.fit(sparse.csr_matrix(matrix).T.tocsr(), show_progress=True)

In [26]:
model = AlternatingLeastSquares(factors=130, regularization=0, 
                                iterations=20, num_threads=16,
                                calculate_training_loss=True)
model.fit(sparse.csr_matrix(matrix).T.tocsr(), show_progress=False)

In [317]:
model = BayesianPersonalizedRanking(factors=50, regularization=0.0001, 
                                    iterations=20, num_threads=16)
model.fit(sparse.csr_matrix(matrix).T.tocsr(), show_progress=False)

### С LightFM немного отдельная история, с подготовкой user-features и item-features

In [7]:
user_features = pd.read_sql('select * from karusel.user_features', db.engine)
user_features['crd_no'] = user_features['crd_no'].astype(str)
user_features = user_features.set_index('crd_no')
user_features = user_features[[x for x in user_features.columns if 'date' not in x and 'cfo_cnt' not in x]]
user_features = user_features[user_features.index.isin(matrix.index)]
user_features = user_features.astype(float).fillna(0)
user_features=(user_features-user_features.min())/(user_features.max()-user_features.min())

item_features = pd.read_sql('select * from karusel.item_features_lvl2', db.engine)
item_features['plu_id'] = item_features['plu_id'].astype(int)
item_features = item_features.set_index('plu_id')
item_features = item_features[[x for x in item_features.columns if 'date' not in x and 'level_2_id' not in x]]
item_features = item_features[item_features.index.isin(matrix.columns)]
item_features = item_features.astype(float).fillna(0)

In [ ]:
from LightFMRecommender.LFMWrapper import WLightFM # Это враппер унаследованный от lightfm.LightFM

model = WLightFM(loss='warp', no_components=70, learning_rate=0.05)
model.fit_data(pd.DataFrame(matrix), user_features=pd.DataFrame(matrix), item_features=pd.DataFrame(matrix).T)
# model.fit_data(matrix)

In [ ]:
# model.set_params(loss='warp', no_components=70)
model.fit(epochs=1, num_threads=12, verbose=True)

### Проверяем метрики

In [32]:
%%time
# fz = fuzzy(matrix, model, val_facts_dict, plu_category_dict, weight_by_price=False, num_recommendations=5, recommend_all=False)
# print('Fuzzy: %s' % fz)
prc = precision(matrix, model, val_facts_dict, weight_by_price=False, num_recommendations=50, recommend_all=False, filter_already_liked=True)
print('Precision: %s' % prc)
# fz_w = fuzzy(matrix, model, val_facts_dict, plu_category_dict, plu_price=plu_price, num_recommendations=5, recommend_all=False)
# print('Fuzzy Weighted: %s' % fz_w)
# prc_w = precision(matrix, model, val_facts_dict, plu_price=plu_price, num_recommendations=5, recommend_all=False)
# print('Precision Weighted: %s' % prc_w)

Precision: 0.017244348762109795
CPU times: user 41.1 s, sys: 6.95 s, total: 48.1 s
Wall time: 48.1 s


### Делаем рекоммендации и считаем ожидаемую выручку

In [179]:
bills = pd.read_sql('select "CRD_NO" as crd_no, "PLU_ID"as plu_id, dates from train_with_promo', db.engine)
bills = bills.merge(products, on='plu_id', how='left')
prices = pd.read_sql('select avg_price, avg_quantity, "PLU_ID"::integer as plu_id '
                     'from karusel.plu_price_avg_count', db.engine)
prd = products[products['plu_id'].isin(matrix.T.index)]
id_plu_dict = matrix.T.reset_index()['index'].to_dict()
plu_name_dict = products.set_index('plu_id').to_dict()['plu_name']
price_df = pd.DataFrame(list(plu_price.items()), columns=['plu_id', 'price'])
prd = prd.merge(price_df, on='plu_id', how='left')

In [180]:
from ALSRecommender.recommendation import RecSys

rs = RecSys(model, matrix)
df = rs.make_recommendations(bills, products, prices, week_threshold=1, num_recommendations=5, filter_already_liked=False, group_seasonality_column='level_2_name')

### Разбиваем по каналам связи

In [252]:
from ALSRecommender.recommendation import Recommendations

category_column = 'level_4_name'

recs = Recommendations(df, 
                       products[['plu_id', 'plu_name', category_column]],
                       user_column_name='crd_no',
                       good_id_column_name='plu_id', good_name_column_name='plu_name')

recs = recs.filter_inactive_cards(bills[['crd_no', 'dates']])
recs = recs.filter_private(private_label_names=['КР.ГОД', 'УМН.РЕШ'])
recs = recs.filter_price(min_price=20, price_column='avg_price')
recs = recs.filter_categories(bills[['crd_no', category_column]], category_column=category_column)
group_1, group_2, group_3 = recs.recommend_for_groups(e_form_factor=0.6, category_column=category_column)

In [253]:
print('Group 1 shape: %s' % group_1.crd_no.nunique())
print('Group 2 shape: %s' % group_2.crd_no.nunique())
print('Group 3 shape: %s' % group_3.crd_no.nunique())

Group 1 shape: 5462
Group 2 shape: 2655
Group 3 shape: 10131


In [191]:
def get_similar_items(model, plu_id, id_plu_dict, plu_name_dict, N=10):
    inv_id_plu_dict = {v: k for k, v in id_plu_dict.items()}
    sim = model.similar_items(inv_id_plu_dict[plu_id], N=N)
    sim_df = pd.DataFrame(sim, columns=['id', 'proba'])
    sim_df['name'] = sim_df['id'].map(lambda x: plu_name_dict[id_plu_dict[x]])
    return sim_df

In [199]:
prd

,level_1_id,level_1_name,level_2_id,level_2_name,level_3_id,level_3_name,level_4_id,level_4_name,plu_id,plu_name,netto_weight,producer,quantity,quantity_no_measure,turnover_with_nds,turnover_no_nds,nds,plu_type,price_category,price
0,CD,Сопутствующие товары,C1,ФК_Бытовая химия,FD0901000,Ароматизаторы,FD0901001,Бытовые осв. воздуха,20712,GLADE Освеж.аэр.морской 300мл,"0,300 КГ",ЭйСиДжонсон,18%,Q,C1_AROMATIZATORY_GLAIDE_300,None,None,None,None,64.12500
1,CD,Сопутствующие товары,C1,ФК_Бытовая химия,FD0901000,Ароматизаторы,FD0901001,Бытовые осв. воздуха,31976,CHIRTON Освежитель воздуха ЛАНДЫШ 300мл,"0,300 КГ",Грейт,18%,Q,C1_AROMATIZATORY_CHIRTON_300,None,None,None,None,49.78125
2,CD,Сопутствующие товары,C1,ФК_Бытовая химия,FD0901000,Ароматизаторы,FD0901001,Бытовые осв. воздуха,38668,GLADE Осв.воз.ПОСЛЕ ДОЖДЯ 300мл,"0,300 КГ",ЭйСиДжонсон,18%,Q,C1_AROMATIZATORY_GLAIDE_300,None,None,None,None,64.31250
3,CD,Сопутствующие товары,C1,ФК_Бытовая химия,FD0901000,Ароматизаторы,FD0901001,Бытовые осв. воздуха,62943,GLADE Освеж.гель цвет.цитр.150г,"0,150 КГ",ЭйСиДжонсон,18%,Q,C1_AROMATIZATORY_GLADE_150,None,None,None,None,107.06250
4,CD,Сопутствующие товары,C1,ФК_Бытовая химия,FD0901000,Ароматизаторы,FD0901001,Бытовые осв. воздуха,62947,GLADE Освеж.аэр.цвет.цитр.300мл,"0,300 КГ",ЭйСиДжонсон,18%,Q,C1_AROMATIZATORY_GLAIDE_300,None,None,None,None,65.43750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16990,GR,Бакалея,GK,ФК_Корма для животны,FD0704000,Корма для собак,FD0704003,Сухой корм для cобак,3461147,"Корм Педигри д/собак кр.пор.Говяд.2,2кг","2,200 КГ",Марс,15%,Q,GK_DOG_SUHOJ_KORM_PEDIGREE_2200,None,None,None,None,254.50000
16991,GR,Бакалея,GK,ФК_Корма для животны,FD0704000,Корма для собак,FD0704003,Сухой корм для cобак,3488661,"PEDIG.Корм сух.д/соб.мин.пор.гов.1,2кг","1,200 КГ",Марс,16%,Q,GK_DOG_SUHOJ_KORM_PEDIGREE_1200,None,None,None,None,166.50000
16992,GR,Бакалея,GK,ФК_Корма для животны,FD0704000,Корма для собак,FD0704003,Сухой корм для cобак,3637122,PERF.FIT Корм с кур.сух.д/соб.мел.п.500г,"0,500 КГ",Марс,18%,Q,GK_SUHOY_KORM_PERF_FIT_500,None,None,None,None,114.37500
16993,GR,Бакалея,GK,ФК_Корма для животны,FD0704000,Корма для собак,FD0704003,Сухой корм для cобак,3637123,PERF.FIT Корм с кур.сух.д/щен.мел.п.500г,"0,500 КГ",Марс,18%,Q,GK_SUHOY_KORM_PERF_FIT_500,None,None,None,None,125.37500


In [202]:
get_similar_items(model, plu_id=62947, id_plu_dict=id_plu_dict, plu_name_dict=plu_name_dict, N=20)

,id,proba,name
0,1359,0.073354,GLADE Освеж.аэр.цвет.цитр.300мл
1,5817,0.067157,BREF Средств.чис.СИЛА АКТИВ ЛИМ.СВЕЖ.50г
2,3155,0.066910,COLG.З/паста МАКС БЛЕСК с отб.пл.100мл
3,1476,0.066779,GILLETTE Кассеты VENUS д/бритья 2шт
4,578,0.066774,DERMOPHIL Губн.помада терм.гиг.3.5г
5,10135,0.066530,ПЕМОЛЮКС Порошок ОСЛЕП.БЕЛЫЙ чист.480г
6,2785,0.066436,NATUR.Проклад.УЛ.МАКСИ 16шт
7,4369,0.066279,GARNIER Дез-ант.MINER.АКТ.КОНТ.рол.50мл
8,11872,0.066246,SAN.CL.Магнит на холодильник
9,8044,0.066228,BREF Средство BLUE AKTIVE чист.д/ун.50г
